In [ ]:
#%pip install vaex
#%pip install nbformat
#%pip install nbconvert
#%pip install seaborn
# Remove comments if libraries are not installed

In [1]:
import numpy as np # type: ignore
import os
import vaex # type: ignore
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def run_notebook(notebook_path):
    # Load the notebook
    with open(notebook_path) as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Execute the notebook
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    ep.preprocess(notebook, {'metadata': {'path': './'}})

# Run the other notebook
run_notebook("data_extraction_to_csv.ipynb")

In [3]:
# Get today's date
today = datetime.today()

# Check if today is Sunday (weekday() returns 6 for Sunday)
if today.weekday() != 6:
    # Subtract the necessary number of days to get to the previous Sunday
    today = today - timedelta(days=(today.weekday() + 1))

# Format the date as 'ddmmyyyy'
today = today.strftime('%d%m%Y')
print(today)

02032025


In [4]:
# set the route directory
csv_p = Path()
csv_folder_path = csv_p / f"Windsor Transit Data Weekly/{today}_transit_windsor"

# read multiple csv's
csv_data = {}
for file in csv_folder_path.iterdir():
    print(file.name)
    csv_name = file.name
    full_csv_path = file.resolve()
    csv_data[csv_name[:-4]] = pd.read_csv(full_csv_path)

# data access
stop_time_data = csv_data['stop_times']
stop_data = csv_data['stops']
routes_data = csv_data['routes']
trips_data = csv_data['trips']

agency.csv
calendar.csv
calendar_dates.csv
fare_attributes.csv
fare_rules.csv
feed_info.csv
routes.csv
shapes.csv
stops.csv
stop_times.csv
trips.csv


In [5]:
def extract_route_data():
    # Step 2.
    expected = routes_data[['route_id', 'route_long_name']]

    # Step 3.
    # Perform the left join
    expected = expected.merge(trips_data, on='route_id', how='left')[['direction_id', 'trip_id','route_long_name', 'route_id']]

    # Step 4.
    expected = expected.merge(stop_time_data, on='trip_id', how='left')[['route_long_name', 'direction_id', 'arrival_time', 
                                                                         'stop_id', 'stop_sequence', 'trip_id', 'departure_time', 'route_id']]

    # Step 5.
    expected = expected.merge(stop_data, on='stop_id', how='left')[['route_id','route_long_name', 'stop_name', 'stop_code','stop_lat', 
                                                                    'stop_lon', 'arrival_time', 'departure_time', 
                                                                    'stop_sequence','trip_id', 'direction_id']]
    expected = expected.sort_values(by=['route_id','trip_id','stop_sequence'])
    expected = expected.drop(columns='route_id')
    return expected

expected = extract_route_data()
expected.head()

,route_long_name,stop_name,stop_code,stop_lat,stop_lon,arrival_time,departure_time,stop_sequence,trip_id,direction_id
13534,TRANSWAY 1A,Windsor Transit Terminal,1000,42.317486,-83.043368,17:50:00,17:50:00,1,1165031,1
13535,TRANSWAY 1A,Ouellette at Chatham,1002,42.317882,-83.039374,17:52:00,17:52:00,2,1165031,1
13536,TRANSWAY 1A,Ouellette at Park,1004,42.315935,-83.038091,17:53:00,17:53:00,3,1165031,1
13537,TRANSWAY 1A,Ouellette at Wyandotte,1006,42.313499,-83.036628,17:54:00,17:54:00,4,1165031,1
13538,TRANSWAY 1A,Ouellette at Elliott,1008,42.311320,-83.035171,17:59:00,17:59:00,5,1165031,1


In [6]:
expected['route_long_name'].unique() 

array(['TRANSWAY 1A', 'TRANSWAY 1C', 'CROSSTOWN 2', 'CENTRAL 3',
       'OTTAWA 4', 'DOUGALL 6', 'WALKERVILLE 8', 'PARENT 14', 'TUNNEL',
       'SOUTH WINDSOR 7', 'LAUZON 10', 'LASALLE 25', 'LTW ROUTE 42',
       '518 EXPRESS', 'ROUTE 605', '418 EXPRESS', 'ROUTE 115',
       'ROUTE 305'], dtype=object)

In [8]:
# Folder name containing the CSV files
folder_name = "12 Months APC Data - Transit Windsor"

# Initialize an empty dictionary to store DataFrames
dfs_dict = {}

# Loop through all files in the folder
for file_name in os.listdir(folder_name):
    if file_name.endswith(".csv"):
        # Extract the month and year from the file name
        file_parts = file_name.split("-")  # Split by "-"
        month_year = file_parts[-1].strip().replace(".csv", "")  # Get "Month Year" part

        # Load the CSV file into a Vaex DataFrame
        file_path = os.path.join(folder_name, file_name)
        df = vaex.open(file_path)

        # Store the DataFrame in the dictionary with the key as "month_year"
        dfs_dict[month_year] = df

# Print keys of the dictionary to verify
print("Loaded DataFrames for:", *list(dfs_dict.keys()), sep='\n')


Loaded DataFrames for:
April 2024
August 2024
December 2023
Feb 2024
Jan 2024
July 2024
June 2024
March 2024
May 2024
November 2023
October 2023
October 2024
September 2024


In [9]:
dfs_dict['September 2024'].head(5)

#,Stop Code,Stop Name,Day of Week,Direction,Month,Passengers In (bin),Passengers Out (bin),Pattern Name,Route Name,Transit Date,Transit Hour,Trip In Filter,Trip Out Filter,Trip Start Transit Hour,TripType,Vehicle,Vehicle Name Filter,ActualTime,Daily Trip ID,Start Trip Time,Number of Trips,Passengers In,Passengers Out,Trip Passengers In,Trip Passengers Out
0,2232,Heritage at Sixth Concession,Tuesday,EASTBOUND,September,1,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:10:34 PM,4474288,2024-09-03 11:54:25 AM,1,1,0,2,5
1,2168,Vollmer Centre Front Entrance,Tuesday,EASTBOUND,September,1,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,11,True,True,11,Regular,8024,True,2024-09-03 11:54:25 AM,4474288,2024-09-03 11:54:25 AM,1,1,0,2,5
2,2263,St Clair College Front Entrance,Tuesday,EASTBOUND,September,0,5,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:16:00 PM,4474288,2024-09-03 11:54:25 AM,1,0,5,2,5
3,2233,Sixth Concession at Montgomery,Tuesday,EASTBOUND,September,0,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:11:40 PM,4474288,2024-09-03 11:54:25 AM,1,0,0,2,5
4,2231,Heritage at Blackthorn,Tuesday,EASTBOUND,September,0,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:09:54 PM,4474288,2024-09-03 11:54:25 AM,1,0,0,2,5


In [10]:
dfs_dict['April 2024']['Route Name'].unique()

['702',
 'WALKERVILLE 8',
 'PARENT 14',
 '755',
 'LASALLE 25',
 '754',
 '706',
 '756',
 '707',
 'TRANSWAY 1C',
 '705',
 'TRANSWAY 1A',
 '708',
 'SOUTH WINDSOR 7',
 '709',
 '418 EXPRESS',
 'OTTAWA 4',
 'ROUTE 605',
 'DOMINION 5',
 '760',
 'TUNNEL',
 'DOUGALL 6',
 '711',
 '710',
 '758',
 'CROSSTOWN 2',
 'LTW ROUTE 42',
 'LAUZON 10',
 'CENTRAL 3',
 '751',
 '518 EXPRESS']

# Change in all the dfs DOMINION 5 for ROUTE 115

In [11]:
for key, df in dfs_dict.items():
    df['Route Name'] = df['Route Name'].apply(lambda x: 'ROUTE 115' if x == 'DOMINION 5' else x)

# Remove not useful columns

* Day of week, bins, transit hour, transit date, trip in, trip out, trip start transit hour, triptype, vehicle filter, number of trips, trip pass in, trip pass out

In [12]:
for key, _ in dfs_dict.items():
    # Use the `drop` method in Vaex to drop the columns
    if 'Day of Week' in dfs_dict[key].column_names:
        dfs_dict[key] = dfs_dict[key].drop(['Day of Week', 'Passengers In (bin)', 'Passengers Out (bin)', 'Transit Date', 'Transit Hour',
                                            'Trip In Filter', 'Trip Out Filter', 'Trip Start Transit Hour', 'Month',
                                            'Vehicle Name Filter', 'Number of Trips', 'Trip Passengers In', 'Trip Passengers Out',
                                            'Pattern Name'])

dfs_dict['March 2024'].head(5)

#,Stop Code,Stop Name,Direction,Route Name,TripType,Vehicle,ActualTime,Daily Trip ID,Start Trip Time,Passengers In,Passengers Out
0,2261,Devonshire Mall at Cineplex,NORTHBOUND,TRANSWAY 1A,Regular,8054,2024-03-22 4:31:25 PM,4220675,2024-03-22 4:31:25 PM,21,0
1,1046,Howard at Grand Marais,NORTHBOUND,TRANSWAY 1A,Regular,8054,2024-03-22 4:38:49 PM,4220675,2024-03-22 4:31:25 PM,3,0
2,1042,Howard at Edinborough,NORTHBOUND,TRANSWAY 1A,Regular,8054,2024-03-22 4:39:54 PM,4220675,2024-03-22 4:31:25 PM,2,1
3,1039,Howard at Hildegard,NORTHBOUND,TRANSWAY 1A,Regular,8054,2024-03-22 4:41:21 PM,4220675,2024-03-22 4:31:25 PM,2,0
4,1041,Howard at McDougall,NORTHBOUND,TRANSWAY 1A,Regular,8054,2024-03-22 4:40:36 PM,4220675,2024-03-22 4:31:25 PM,1,0


# Calculate Passenger Transfer

In [13]:
for key, df in dfs_dict.items():
    df['Passengers Transfer'] = df['Passengers In'] - df['Passengers Out']

# Add Actual Bus Occupancy column

In [14]:
def net_passenger(data):
    # Convert Vaex DataFrame to Pandas DataFrame
    data = data.to_pandas_df()

    # Sort the DataFrame by 'Route Name', 'Daily Trip ID', and 'ActualTime'
    data = data.sort_values(by=['Route Name', 'Daily Trip ID', 'ActualTime'])

    # Calculate 'Passengers Transfer'
    data['Passengers Transfer'] = data['Passengers In'] - data['Passengers Out']

    # Calculate cumulative sum of 'Passengers Transfer' for each 'Route Name' and 'Daily Trip ID'
    data['Actual Bus Occupancy'] = data.groupby(['Route Name', 'Daily Trip ID'])['Passengers Transfer'].cumsum()

    # Group the data once to avoid repeated filtering inside the loop
    grouped = data.groupby('Daily Trip ID', group_keys=False)

    # Process each group
    def process_trip(trip_data):
        # Get final occupancy
        final_occupancy = trip_data['Actual Bus Occupancy'].iloc[-1]
        passengers_transfer = trip_data['Passengers Transfer'].values
        occupancy_cumsum = trip_data['Actual Bus Occupancy'].values

        # Adjust based on final occupancy
        if final_occupancy < 0:
            # Find minimum negative value and adjust the first occurrence
            min_value = occupancy_cumsum.min()
            first_negative_idx = occupancy_cumsum.argmin()  # Find position in the array
            passengers_transfer[first_negative_idx] += abs(min_value)

        elif final_occupancy > 0:
            # Find the first occurrence of maximum occupancy and adjust
            max_value = occupancy_cumsum.max()
            first_max_idx = occupancy_cumsum.argmax()  # Find position in the array
            passengers_transfer[first_max_idx] -= final_occupancy

        # Update adjusted cumulative sum
        adjusted_cumsum = passengers_transfer.cumsum()

        # Ensure no negative occupancy values remain
        while (adjusted_cumsum < 0).any():
            min_value = adjusted_cumsum.min()
            first_negative_idx = (adjusted_cumsum < 0).argmax()  # Find position in the array
            passengers_transfer[first_negative_idx] += abs(min_value)
            adjusted_cumsum = passengers_transfer.cumsum()

        # Update the DataFrame with the adjusted values
        trip_data['Passengers Transfer'] = passengers_transfer
        trip_data['Adjusted Bus Occupancy'] = adjusted_cumsum

        return trip_data

    # Apply the processing function to each group and collect the results
    data = grouped.apply(process_trip, include_groups=False)

    # Replace Actual Bus Occupancy with the adjusted and remove the temporary column
    data['Actual Bus Occupancy'] = data['Adjusted Bus Occupancy']
    data = data.drop(columns=['Adjusted Bus Occupancy'])

    # Convert back to Vaex DataFrame
    data_vaex = vaex.from_pandas(data)

    return data_vaex


In [15]:
i = 1
for key, df in dfs_dict.items():
    print(f'Processing: {key} data ({i})')
    i += 1
    dfs_dict[key] = net_passenger(df)

Processing: April 2024 data (1)
Processing: August 2024 data (2)
Processing: December 2023 data (3)
Processing: Feb 2024 data (4)
Processing: Jan 2024 data (5)
Processing: July 2024 data (6)
Processing: June 2024 data (7)
Processing: March 2024 data (8)
Processing: May 2024 data (9)
Processing: November 2023 data (10)
Processing: October 2023 data (11)
Processing: October 2024 data (12)
Processing: September 2024 data (13)


# Create a daily trip sequence, bus order during the day

In [16]:
def assign_daily_order_trip_id_by_route_time_direction_vaex(data):
    # Convert Vaex DataFrame to Pandas
    df = data.to_pandas_df()

    # Parse datetime explicitly with AM/PM handling
    df['Start Trip Time'] = pd.to_datetime(
        df['Start Trip Time'], 
        format='%Y-%m-%d %I:%M:%S %p',  # Explicitly account for AM/PM
        errors='coerce'  # Handle invalid timestamps gracefully
    )

    # Subtract 4 hours to align trips with the same "Business Day"
    df['Adjusted Start Time'] = df['Start Trip Time'] - timedelta(hours=4)

    # Extract the adjusted date for grouping
    df['Adjusted Date'] = df['Adjusted Start Time'].dt.date  # Keeps datetime.date format

    # Sort by Route Name, Direction, Adjusted Date, and Start Trip Time
    df = df.sort_values(by=['Route Name', 'Direction', 'Adjusted Date', 'Start Trip Time']).reset_index(drop=True)

    # Assign daily_order_trip_id by grouping on Route Name, Direction, Adjusted Date, and Start Trip Time
    df['daily_order_trip_id'] = (
        df.groupby(['Adjusted Date', 'Route Name', 'Direction'])['Start Trip Time']
        .transform('rank', method='dense').astype(int)
    )

    # Rename Adjusted Date to Business Day without changing its format
    df = df.rename({'Adjusted Date': 'Business Day'}, axis=1)

    # Drop temporary columns if not needed
    df = df.drop(columns=['Adjusted Start Time'])
    
    # Convert Business Day to pandas datetime before formatting
    df['Business Day'] = pd.to_datetime(df['Business Day'])  # Convert datetime.date to pandas datetime
    df['Business Day'] = df['Business Day'].dt.strftime('%Y-%m-%d')  # Format as string in 'YYYY-MM-DD'

    # Convert back to Vaex DataFrame
    df = vaex.from_pandas(df)
    df['Start Trip Time'] = df['Start Trip Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df



In [17]:
for key, df in dfs_dict.items():
    print(f'Processing: {key} data')
    dfs_dict[key] = assign_daily_order_trip_id_by_route_time_direction_vaex(df)

Processing: April 2024 data
Processing: August 2024 data
Processing: December 2023 data
Processing: Feb 2024 data
Processing: Jan 2024 data
Processing: July 2024 data
Processing: June 2024 data
Processing: March 2024 data
Processing: May 2024 data
Processing: November 2023 data
Processing: October 2023 data
Processing: October 2024 data
Processing: September 2024 data


In [18]:
#dfs_dict['March 2024'].head(5)

# Convert Actual Time column to be in the 24H format

In [19]:
def convert_to_24h(df):
    # Convert Vaex DataFrame to Pandas DataFrame
    pandas_df = df.to_pandas_df()
    
    # Ensure 'ActualTime' is datetime and convert to 24-hour format
    pandas_df['ActualTime'] = pd.to_datetime(pandas_df['ActualTime'], format='%Y-%m-%d %I:%M:%S %p', errors='coerce')
    pandas_df['ActualTime'] = pandas_df['ActualTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Convert back to Vaex DataFrame
    vaex_df = vaex.from_pandas(pandas_df)
    return vaex_df


In [20]:
for key, df in dfs_dict.items():
    print(f'Processing: {key} data')
    dfs_dict[key] = convert_to_24h(df)

Processing: April 2024 data
Processing: August 2024 data
Processing: December 2023 data
Processing: Feb 2024 data
Processing: Jan 2024 data
Processing: July 2024 data
Processing: June 2024 data
Processing: March 2024 data
Processing: May 2024 data
Processing: November 2023 data
Processing: October 2023 data
Processing: October 2024 data
Processing: September 2024 data


In [21]:
dfs_dict['December 2023'].head()

#,Stop Code,Stop Name,Direction,Route Name,TripType,Vehicle,ActualTime,Start Trip Time,Passengers In,Passengers Out,Passengers Transfer,Actual Bus Occupancy,Business Day,daily_order_trip_id
0,1834,Hotel Dieu Grace Healthcare,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:10:32,2023-12-01 07:10:32,0,0,0,0,2023-12-01,1
1,2090,Prince at College,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:18:39,2023-12-01 07:10:32,1,0,1,1,2023-12-01,1
2,1215,Sandwich at South,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:20:35,2023-12-01 07:10:32,0,0,0,1,2023-12-01,1
3,1219,Sandwich at Brock,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:21:18,2023-12-01 07:10:32,1,0,1,2,2023-12-01,1
4,1222,Wyandotte at Mill,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:22:58,2023-12-01 07:10:32,4,0,4,6,2023-12-01,1
5,1228,Wyandotte at Sunset,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:25:02,2023-12-01 07:10:32,3,0,3,9,2023-12-01,1
6,2345,California and College,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:26:23,2023-12-01 07:10:32,3,0,3,12,2023-12-01,1
7,2346,College at Campbell,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:28:17,2023-12-01 07:10:32,6,0,6,18,2023-12-01,1
8,1421,Crawford at College,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:30:23,2023-12-01 07:10:32,1,1,0,18,2023-12-01,1
9,1414,Crawford at Tecumseh,EASTBOUND,418 EXPRESS,Regular,8031,2023-12-01 07:31:46,2023-12-01 07:10:32,1,0,1,19,2023-12-01,1


# Remove rows that are not in the direction map

In [22]:
direction_map = {'TRANSWAY 1A': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'},
 'TRANSWAY 1C': {1: 'EASTBOUND', 0: 'WESTBOUND'},
 'CROSSTOWN 2': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'CENTRAL 3': {1: 'EASTBOUND', 0: 'WESTBOUND'},
 'OTTAWA 4': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'DOUGALL 6': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'WALKERVILLE 8': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'PARENT 14': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'},
 'TUNNEL': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'SOUTH WINDSOR 7': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'LAUZON 10': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'LASALLE 25': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'LTW ROUTE 42': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 '518 EXPRESS': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'ROUTE 605': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'},
 '418 EXPRESS': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'ROUTE 115': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'}}

In [23]:
routes = [route for route in direction_map.keys()]

In [24]:
for key, df in dfs_dict.items():
    dfs_dict[key] = df[df['Route Name'].isin(routes)]

# Change direction column
* `'NORTHBOUND'` -> `'N'`
* `'SOUTHBOUND'` -> `'S'`
* `'WESTBOUND'` -> `'w'`
* `'EASTOUND'` -> `'E'`

In [25]:
def process_direction(df):
    direction_mapping = {
    "NORTHBOUND": "N",
    "SOUTHBOUND": "S",
    "WESTBOUND": "W",
    "EASTBOUND": "E",
    "WEST":"W",
    "EAST":'E'}
    df["Direction"] = df.Direction.map(direction_mapping)
    return df

In [26]:

i = 1
for month, df in dfs_dict.items():
    print(f'Processing {month} ({i})')
    dfs_dict[month] = process_direction(df)
    i =+ 1

Processing April 2024 (1)
Processing August 2024 (1)
Processing December 2023 (1)
Processing Feb 2024 (1)
Processing Jan 2024 (1)
Processing July 2024 (1)
Processing June 2024 (1)
Processing March 2024 (1)
Processing May 2024 (1)
Processing November 2023 (1)
Processing October 2023 (1)
Processing October 2024 (1)
Processing September 2024 (1)


# Concatenate the latitude and longitud columns to the datasets

In [27]:
expected.head(5)

,route_long_name,stop_name,stop_code,stop_lat,stop_lon,arrival_time,departure_time,stop_sequence,trip_id,direction_id
13534,TRANSWAY 1A,Windsor Transit Terminal,1000,42.317486,-83.043368,17:50:00,17:50:00,1,1165031,1
13535,TRANSWAY 1A,Ouellette at Chatham,1002,42.317882,-83.039374,17:52:00,17:52:00,2,1165031,1
13536,TRANSWAY 1A,Ouellette at Park,1004,42.315935,-83.038091,17:53:00,17:53:00,3,1165031,1
13537,TRANSWAY 1A,Ouellette at Wyandotte,1006,42.313499,-83.036628,17:54:00,17:54:00,4,1165031,1
13538,TRANSWAY 1A,Ouellette at Elliott,1008,42.311320,-83.035171,17:59:00,17:59:00,5,1165031,1


In [28]:
df = dfs_dict['March 2024'].copy()
df.head(5)

#,Stop Code,Stop Name,Direction,Route Name,TripType,Vehicle,ActualTime,Start Trip Time,Passengers In,Passengers Out,Passengers Transfer,Actual Bus Occupancy,Business Day,daily_order_trip_id
0,1834,Hotel Dieu Grace Healthcare,E,418 EXPRESS,Regular,8069,2024-03-01 06:41:52,2024-03-01 06:41:52,1,0,1,1,2024-03-01,1
1,2090,Prince at College,E,418 EXPRESS,Regular,8069,2024-03-01 06:46:41,2024-03-01 06:41:52,0,0,0,1,2024-03-01,1
2,1215,Sandwich at South,E,418 EXPRESS,Regular,8069,2024-03-01 06:48:27,2024-03-01 06:41:52,1,0,1,2,2024-03-01,1
3,1219,Sandwich at Brock,E,418 EXPRESS,Regular,8069,2024-03-01 06:49:29,2024-03-01 06:41:52,1,0,1,3,2024-03-01,1
4,1222,Wyandotte at Mill,E,418 EXPRESS,Regular,8069,2024-03-01 06:51:03,2024-03-01 06:41:52,4,0,4,7,2024-03-01,1


In [29]:
expected = expected[expected['route_long_name'].isin(routes)]
unique_stop_codes = expected.drop_duplicates(subset=['stop_code'], keep='first')[['stop_code', 'stop_lat', 'stop_lon']]
unique_stop_codes = vaex.from_pandas(unique_stop_codes)
unique_stop_codes.head(5)

#,stop_code,stop_lat,stop_lon
0,1000,42.3175,-83.0434
1,1002,42.3179,-83.0394
2,1004,42.3159,-83.0381
3,1006,42.3135,-83.0366
4,1008,42.3113,-83.0352


In [ ]:
# Perform the join operation
for key, df in dfs_dict.items():
    dfs_dict[key] = df.join(
        unique_stop_codes,          # The right DataFrame
        left_on='Stop Code',        # Column in the left DataFrame (df)
        right_on='stop_code',       # Column in the right DataFrame (unique_stop_codes)
        how='left'                  # Perform a left join
        )

dfs_dict['June 2024'].head(5)

In [ ]:
test = dfs_dict['March 2024'].copy()

#test[(test['Route Name']=='TRANSWAY 1A') & (test['daily_order_trip_id']==1)].head()

# Add Stop Sequence Column

In [ ]:
def assign_stop_sequence(df):
    # Convert Vaex DataFrame to Pandas DataFrame
    df_pandas = df.to_pandas_df()
    
    # Ensure ActualTime is in datetime format in Pandas
    df_pandas['ActualTime'] = pd.to_datetime(df_pandas['ActualTime']).dt.floor('s')
    
    # Sort and assign stop sequence using the Pandas logic
    df_pandas = df_pandas.sort_values(by=['Route Name', 'Direction', 'Start Trip Time', 'ActualTime']).reset_index(drop=True)
    df_pandas['stop_sequence'] = (
        df_pandas.groupby(['Route Name', 'Direction', 'Start Trip Time']).cumcount() + 1
    )
    
    # Convert back to Vaex DataFrame
    df_vaex = vaex.from_pandas(df_pandas)

    df_vaex['ActualTime'] = df_vaex['ActualTime'].astype('datetime64').dt.strftime('%Y-%m-%d %H:%M:%S')

    return df_vaex

In [ ]:
i = 1
for month, df in dfs_dict.items():
    print(f'Processing {month}. ({i})')
    i += 1
    dfs_dict[month] = assign_stop_sequence(dfs_dict[month]).copy()

# Add the Expected Arrive Time column to each month

In [ ]:
test = dfs_dict['March 2024'].copy()
test.head(5)

In [ ]:
def calculate_travel_time(df_vaex):
    # Convert Vaex DataFrame to Pandas DataFrame
    df_pandas = df_vaex.to_pandas_df()

    # Ensure ActualTime is in datetime format
    df_pandas['ActualTime'] = pd.to_datetime(df_pandas['ActualTime'])

    # Sort the DataFrame by grouping columns and stop_sequence
    df_pandas = df_pandas.sort_values(by=['Business Day', 'Route Name', 'Direction', 'daily_order_trip_id', 'stop_sequence'])

    # Group by and calculate the travel time
    df_pandas['prev_time'] = df_pandas.groupby(['Business Day', 'Route Name', 'Direction', 'daily_order_trip_id'])['ActualTime'].shift(1)
    df_pandas['travel_time (sec)'] = (df_pandas['ActualTime'] - df_pandas['prev_time']).dt.total_seconds()

    # Replace NaN values with 0
    df_pandas['travel_time (sec)'] = df_pandas['travel_time (sec)'].fillna(0)

    # Drop the intermediate column if not needed
    df_pandas = df_pandas.drop(columns=['prev_time'])

    # Convert back to Vaex DataFrame
    df_vaex = vaex.from_pandas(df_pandas)
    df_vaex['ActualTime'] = df_vaex['ActualTime'].astype('datetime64').dt.strftime('%Y-%m-%d %H:%M:%S')

    return df_vaex

#test2 = calculate_travel_time(test)

In [ ]:
def calculate_expected_travel_time(df_vaex):
    # Convert Vaex DataFrame to Pandas DataFrame
    df_pandas = df_vaex.to_pandas_df()

    # Ensure 'travel_time (sec)' is numeric
    df_pandas['travel_time (sec)'] = pd.to_numeric(df_pandas['travel_time (sec)'], errors='coerce')

    # Filter rows where stop_sequence != 1 and exclude 'travel_time (sec)' == 0
    filtered_df = df_pandas[df_pandas['stop_sequence'] != 1]
    filtered_df = filtered_df[filtered_df['travel_time (sec)'] > 0]

    # Group by 'Route Name', 'Direction', and 'Stop Code' and calculate the minimum 'travel_time (sec)' (non-zero for stop_sequence != 1)
    grouped_df = (
        filtered_df
        .groupby(['Route Name', 'Direction', 'Stop Code'], as_index=False)['travel_time (sec)']
        .min()
    )

    # Add the new column for expected travel time back to the original DataFrame
    df_pandas = df_pandas.merge(
        grouped_df,
        on=['Route Name', 'Direction', 'Stop Code'],
        how='left',
        suffixes=('', '_expected')  # Appends '_expected' to the new column
    )

    # Rename the new column for clarity
    df_pandas.rename(columns={'travel_time (sec)_expected': 'expected_travel_time_sec'}, inplace=True)

    # For rows where stop_sequence == 1, set expected_travel_time_sec to NaN (since the condition excludes stop_sequence == 1)
    df_pandas.loc[df_pandas['stop_sequence'] == 1, 'expected_travel_time_sec'] = float('nan')

    df_pandas['expected_travel_time_sec'] = df_pandas['expected_travel_time_sec'].fillna(0)

    df_pandas = df_pandas.drop(columns=['travel_time (sec)'])

    # Convert back to Vaex DataFrame
    result_df_vaex = vaex.from_pandas(df_pandas)

    return result_df_vaex

#test3 = calculate_expected_travel_time(test2)

In [ ]:
# Calculate cumulative expected travel time per stop
def calculate_cumulative_expected_travel_time(df_vaex):
    # Convert Vaex DataFrame to Pandas DataFrame
    df_pandas = df_vaex.to_pandas_df()

    # Ensure expected_travel_time_sec is numeric
    df_pandas['expected_travel_time_sec'] = pd.to_numeric(df_pandas['expected_travel_time_sec'], errors='coerce')

    # Sort the DataFrame by grouping columns and stop_sequence
    df_pandas = df_pandas.sort_values(by=['Business Day', 'Route Name', 'Direction', 'daily_order_trip_id', 'stop_sequence'])

    # Group by and calculate the cumulative sum for expected_travel_time_sec
    df_pandas['cumulative_expected_travel_time_sec'] = (
        df_pandas
        .groupby(['Business Day', 'Route Name', 'Direction', 'daily_order_trip_id'])['expected_travel_time_sec']
        .cumsum()
    )

    # Drop the expected_travel_time_sec column
    df_pandas = df_pandas.drop(columns=['expected_travel_time_sec'])

    # Convert back to Vaex DataFrame
    result_df_vaex = vaex.from_pandas(df_pandas)

    return result_df_vaex

#test4 = calculate_cumulative_expected_travel_time(test3)

In [ ]:
def calculate_expected_arrival_time(df_vaex):
    # Convert Vaex DataFrame to Pandas DataFrame
    df_pandas = df_vaex.to_pandas_df()

    # Ensure Start Trip Time is in datetime format
    df_pandas['Start Trip Time'] = pd.to_datetime(df_pandas['Start Trip Time'])

    # Add cumulative_expected_travel_time_sec (in seconds) to Start Trip Time
    df_pandas['expected_arrival_time'] = df_pandas['Start Trip Time'] + pd.to_timedelta(df_pandas['cumulative_expected_travel_time_sec'], unit='s')

    # Format Start Trip Time and expected arrival time to only include up to seconds
    df_pandas['Start Trip Time'] = df_pandas['Start Trip Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df_pandas['expected_arrival_time'] = df_pandas['expected_arrival_time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Drop the cumulative_expected_travel_time_sec column
    df_pandas = df_pandas.drop(columns=['cumulative_expected_travel_time_sec'])

    # Convert back to Vaex DataFrame
    result_df_vaex = vaex.from_pandas(df_pandas)

    return result_df_vaex

#test5 = calculate_expected_arrival_time(test4)

# Make the vaex file and export them as csv and also add a month column

In [ ]:
# Iterate over each DataFrame in the dictionary
for key, df in dfs_dict.items():
    # Copy the DataFrame to avoid modifying the original
    df_copy = df.copy()
    
    # Determine the numeric value of the month based on the dictionary key
    month_name = key.split()[0]  # Extract the month part from the key
    month_map = {
        "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
        "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12,
        "Jan": 1, "Feb": 2  # Handle abbreviated month names
    }
    month_number = month_map.get(month_name, 0)  # Default to 0 if the name is not recognized

    # Generate a constant array of the month number matching the unfiltered length
    full_array = np.full(df_copy.length_unfiltered(), month_number, dtype=int)

    # Add a physical column to the DataFrame
    df_copy.add_column('month', full_array)

    # Update the dictionary with the modified DataFrame
    dfs_dict[key] = df_copy

In [ ]:
# Fix direction issue WEST -> WESTBOUND, EAST -> EASTBOUND

def direction_formatting(df):
    # Define the mapping dictionary
    direction_mapping = {
        "WEST": "W",
        "EAST": "E"
    }

    # Apply the mapping using an expression
    df['Direction'] = df['Direction'].apply(lambda x: direction_mapping.get(x, x))

    return df

In [ ]:
test8 = dfs_dict['March 2024'].copy()
direction_formatting(test8)['Direction'].unique()

In [ ]:
i = 1
for month, df in dfs_dict.items():
    print(f'Calculating Expecting Travel Time for {month} ({i})')
    i += 1
    test2 = calculate_travel_time(df)
    test3 = calculate_expected_travel_time(test2)
    test4 = calculate_cumulative_expected_travel_time(test3)
    test5 = direction_formatting(test4)
    dfs_dict[month] = calculate_expected_arrival_time(test5)

In [ ]:
for month, df in dfs_dict.items():
    dfs_dict[month] = df.drop(['stop_code'])

In [ ]:
dfs_dict['March 2024'].head()

In [ ]:
# Define the folder name
folder_name = "cleaned_dataset"

# Create the folder if it doesn't exist
os.makedirs(folder_name, exist_ok=True)
i = 1

# Iterate over the dictionary and save each DataFrame as a CSV
for key, df in dfs_dict.items():
    # Replace spaces in the key with underscores
    file_name = key.replace(" ", "_") + ".csv"
    
    # Define the full path for the CSV
    file_path = os.path.join(folder_name, file_name)
    
    # Save the DataFrame as a CSV
    df.export_csv(file_path)
    
    print(f"Saved {file_path} ({i})")
    i += 1

# Analyze relationship between bus occupancy and current features

## Preprocessing:
* #### Separate date data into different columns (Week number, Week day, hour) and map them with numbers.
NOTE: Do NOT REMOVE the column, maybe a copy of the df that can be edited may help

## Make a heatmap:
* #### Sorted by correlation value (Top to Bottom)
* #### Use absolute value of the correlation to sort them
* #### Bottom labels with numbers to make it easier to read, assign a number to each feature
* #### Friendly colorblind gradient

In [ ]:
df = dfs_dict['September 2024'].copy()
df.head(5)

In [ ]:
# Remove columns that does not make sense to be correlated to the bus occupancy variable.
# Vehicle, Start Trip Time, Stop Code (if we want to measure the correlation with location is better to use latitude and longitude), 
# stop_sequence (there is no diference in the trips so it can be ignored), month (for now because all is from the same month itwont change),
# daily_order_trip_id (this is already related to the time in Start Trip Time column), Start Trip Time (The feature for the time will be ActualTime)
# expected_arrival_time (the team told me to not use it for now), Stop Name (is already related to location variables),
# Passengers In, Passengers Out, Passengers Transfer (these three variables are used to calculate the target variable so is expected tobe directly related)

# Create a column from 'ActualTime' minus 'expected_arrival_time' to estimate the delay and add that to the correlation heatmap

# Features to use: 'Direction','Route Name', 'ActualTime', 'stop_lat', 'stop_sequence', 'Actual Bus Occupancy'


df = df[['Direction','Route Name', 'ActualTime','Business Day','stop_lat', 'stop_lon', 'Actual Bus Occupancy', 'expected_arrival_time']]
#df.head(5)

# Create the column for delay (`ActualTime` - `extepected_arrival_time`)

In [ ]:
def delay_calc(df):

    df = df.to_pandas_df()
    df['ActualTime'] = pd.to_datetime(df['ActualTime'])
    df['expected_arrival_time'] = pd.to_datetime(df['expected_arrival_time'])
    df['delay'] = (df['ActualTime'] - df['expected_arrival_time']).dt.total_seconds()
    df = vaex.from_pandas(df)
    df['expected_arrival_time'] = df['expected_arrival_time'].astype('datetime64').dt.strftime('%Y-%m-%d %H:%M:%S')
    df['ActualTime'] = df['ActualTime'].astype('datetime64').dt.strftime('%Y-%m-%d %H:%M:%S')
    df['Business Day'] = df['Business Day'].astype('datetime64').dt.strftime('%Y-%m-%d')
    
    return df
if 'delay' not in df.get_column_names():
    df = delay_calc(df)

# Create a column for hour based on ActualTime, Weekday number (Sunday = 1, Saturday = 7), Weeknumber (First week = 1)

In [ ]:
def process_datetime_features(df):
    df = df.to_pandas_df()

    # Ensure the time column is in datetime format
    df['ActualTime'] = pd.to_datetime(df['ActualTime'])
    
    # Extract hour from the time column
    df['hour'] = df['ActualTime'].dt.hour

    # Ensure the date column is in datetime format
    df['Business Day'] = pd.to_datetime(df['Business Day'])

    # Calculate weekday number (Sunday = 1, Monday = 2, ..., Saturday = 7)
    df['weekday_num'] = df['Business Day'].dt.dayofweek + 1

    # Calculate week number within the month
    df['week_order'] = (df['Business Day'].dt.day - 1)//7 + 1

    # Convert back to Vaex DataFrame
    df = vaex.from_pandas(df)
    df['ActualTime'] = df['ActualTime'].astype('datetime64').dt.strftime('%Y-%m-%d %H:%M:%S')
    df['Business Day'] = df['Business Day'].astype('datetime64').dt.strftime('%Y-%m-%d')
    return df

temp_df = process_datetime_features(df)

# Create an overcrowded column (0 if Bus Occupancy 35 or less, more than 35 put 1)

In [ ]:
def overcrowd_status(df_vaex):
    # Convert Vaex DataFrame to Pandas
    df_pandas = df_vaex.to_pandas_df()

    # Ensure 'Actual Bus Occupancy' is numeric
    df_pandas['Actual Bus Occupancy'] = pd.to_numeric(df_pandas['Actual Bus Occupancy'], errors='coerce')

    # Apply overcrowding condition (1 if > 35, else 0)
    df_pandas['Overcrowded'] = df_pandas['Actual Bus Occupancy'].apply(lambda x: 1 if x > 35 else 0)

    # Convert back to Vaex DataFrame
    df_vaex_result = vaex.from_pandas(df_pandas)

    return df_vaex_result

# Apply the function
temp_df = overcrowd_status(temp_df)


In [ ]:
selected_features = ['Direction', 'Route Name', 'stop_lat', 'stop_lon', 'hour', 'weekday_num', 'week_order','Actual Bus Occupancy', 'Overcrowded', 'delay']

selected_df = temp_df[selected_features].copy()
selected_df.head(5)

# Encode Direction as a cyclical encoding to better adjust to direction, create two new columns (`dir_hzt`, `dir_vrt`)

In [ ]:
def cycle_encoding(df):
    # Define new columns using efficient Vaex expressions
    df['dir_hzt'] = (
        (df.Direction == 'E') * 1 +
        (df.Direction == 'W') * -1
    )

    df['dir_vrt'] = (
        (df.Direction == 'N') * 1 +
        (df.Direction == 'S') * -1
    )
    return df

selected_df= cycle_encoding(selected_df)

# Heatmap creation

In [ ]:
# Convert Vaex DataFrame to Pandas
df_pandas = selected_df.to_pandas_df().drop(columns='Direction', axis=1)

# Label encode categorical columns
# df_pandas['Direction'] = df_pandas['Direction'].astype('category').cat.codes
df_pandas['Route Name'] = df_pandas['Route Name'].astype('category').cat.codes

# Compute the correlation matrix (absolute values for sorting)
corr_matrix = df_pandas.drop(['Actual Bus Occupancy'], axis=1).corr().abs()

# Sort correlation matrix by highest correlation with 'Overcrowded'
sorted_corr = corr_matrix['Overcrowded'].sort_values(ascending=False).index
corr_matrix = corr_matrix.loc[sorted_corr, sorted_corr]

# Generate numerical labels only
num_labels = [str(i) for i in range(len(corr_matrix.columns))]

# Plot the heatmap
plt.figure(figsize=(10, 6))
ax = sns.heatmap(corr_matrix, annot=True, cmap="viridis", fmt=".4f", linewidths=0.5)

# Set numerical labels for easier reading
ax.set_xticklabels(num_labels, rotation=0)  # Horizontal labels
ax.set_yticklabels([f"{i}. {col}" for i, col in enumerate(corr_matrix.index)], rotation=0)

plt.title("Correlation Heatmap (Bus Status / Binary)", fontsize=14)
plt.ylabel("Features")
plt.show()

In [ ]:
# Convert Vaex DataFrame to Pandas
df_pandas = selected_df.to_pandas_df().drop(columns='Direction', axis=1)

# Label encode categorical columns
# df_pandas['Direction'] = df_pandas['Direction'].astype('category').cat.codes
df_pandas['Route Name'] = df_pandas['Route Name'].astype('category').cat.codes

# Compute the correlation matrix (absolute values for sorting)
corr_matrix = df_pandas.drop(['Overcrowded'], axis=1).corr().abs()

# Sort correlation matrix by highest correlation with 'Actual Bus Occupancy'
sorted_corr = corr_matrix['Actual Bus Occupancy'].sort_values(ascending=False).index
corr_matrix = corr_matrix.loc[sorted_corr, sorted_corr]

# Generate numerical labels only
num_labels = [str(i) for i in range(len(corr_matrix.columns))]

# Plot the heatmap
plt.figure(figsize=(10, 6))
ax = sns.heatmap(corr_matrix, annot=True, cmap="viridis", fmt=".4f", linewidths=0.5)

# Set numerical labels for easier reading
ax.set_xticklabels(num_labels, rotation=0)  # Horizontal labels
ax.set_yticklabels([f"{i}. {col}" for i, col in enumerate(corr_matrix.index)], rotation=0)

plt.title("Correlation Heatmap (Bus Occupancy / Discrete)", fontsize=14)
plt.ylabel("Features")
plt.show()